<a href="https://colab.research.google.com/github/KOreshOK9922/distances/blob/main/13082024_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from geopy.geocoders import Nominatim #Подключаем библиотеку
from geopy.distance import geodesic #И дополнения

geolocator = Nominatim(user_agent="Tester") #Указываем название приложения
address_1 = str(input('Введите город 1: \n')) #Получаем название первого города
address_2 = str(input('Введите город 2: \n')) #Получаем название второго города
location_1 = geolocator.geocode(address_1) #Получаем полное название первого города
location_2 = geolocator.geocode(address_2) #Получаем полное название второго города
print('Город 1: ', location_1) #Выводим первичные данные
print('Город 2: ', location_2) #Выводим первичные данные
print('Координаты города 1: ', location_1.latitude, location_1.longitude) #Выводим координаты первого города
gps_point_1 = location_1.latitude, location_1.longitude #Выводим координаты первого города
gps_point_2 = location_2.latitude, location_2.longitude #Выводим координаты второго города
print('Координаты города 2: ', location_2.latitude, location_2.longitude) #Выводим общие данные
print('Дистанция между городом', location_1, 'и городом ', location_2, ': ', geodesic(gps_point_1, gps_point_2).kilometers, ' км') #Выводим полученный результат в километрах

Введите город 1: 
СаМаРа
Введите город 2: 
ВОЛгоград
Город 1:  Самара, городской округ Самара, Самарская область, Приволжский федеральный округ, 443028, Россия
Город 2:  Волгоград, городской округ Волгоград, Волгоградская область, Южный федеральный округ, 400066, Россия
Координаты города 1:  53.2194648 50.2039316
Координаты города 2:  48.7081906 44.5153353
Дистанция между городом Самара, городской округ Самара, Самарская область, Приволжский федеральный округ, 443028, Россия и городом  Волгоград, городской округ Волгоград, Волгоградская область, Южный федеральный округ, 400066, Россия :  641.1253630489408  км


In [37]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from geopy.distance import geodesic
import time
from requests.exceptions import ReadTimeout

# Загрузка данных из файла Excel
try:
    df = pd.read_excel('/content/point.xlsx')
    print("Файл Excel успешно загружен.")
    print("Имена всех столбцов в DataFrame:")
    for column in df.columns:
        print(column)
except Exception as e:
    print(f"Ошибка при загрузке файла Excel: {e}")

# Инициализация геолокатора Nominatim и RateLimiter
geolocator = Nominatim(user_agent="Tester")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=3)

# Функция для очистки адреса перед геокодированием
def clean_address(address):
    return " ".join(address.split())

# Функция для геокодирования с обработкой исключений
def custom_geocode(address):
    location = None
    while location is None:
        try:
            location = geocode(clean_address(address), timeout=5)
            return location
        except ReadTimeout:
            print("Таймаут чтения. Повторная попытка...")
            time.sleep(1)
        except Exception as e:
            print(f"Ошибка геокодирования адреса: {address}. {e}")
            return None
    return location

# Процесс обработки адресов и расчета расстояний
print("\nОбработка адресов и расчет расстояний...")

for index, row in df.iterrows():
    address = row['Адрес']
    location = custom_geocode(address)

    if location is not None:
        df.at[index, 'Широта'] = location.latitude
        df.at[index, 'Долгота'] = location.longitude
        print(f"Координаты для {address}: {location.latitude}, {location.longitude}")
    else:
        print(f"Координаты для {address} не найдены.")
        df.at[index, 'Расстояние'] = None

    time.sleep(3)

distances = []

for i in range(len(df)-1):
    if pd.notna(df.at[i, 'Широта']) and pd.notna(df.at[i+1, 'Широта']):
        distance = geodesic((df.at[i, 'Широта'], df.at[i, 'Долгота']), (df.at[i+1, 'Широта'], df.at[i+1, 'Долгота'])).kilometers
        df.at[i, 'Расстояние'] = distance
        print(f"Расстояние между {df.at[i, 'Адрес']} и {df.at[i+1, 'Адрес']}: {distance:.2f} км")
        distances.append(distance)

print("\nВсе расстояния:")
for i, dist in enumerate(distances):
    print(f"Расстояние {i+1}: {dist:.2f} км")

# Сохранение результатов обратно в файл Excel
try:
    df.to_excel('/content/point_updated.xlsx', index=False)
    print("\nОбновленные данные сохранены в файл Excel.")
except Exception as e:
    print(f"Ошибка сохранения в файл Excel: {e}")

Файл Excel успешно загружен.
Имена всех столбцов в DataFrame:
Адрес
Широта
Долгота
Расстояние

Обработка адресов и расчет расстояний...
Координаты для  Красный Яр сп Комсомольская 82 не найдены.
Координаты для  Самара Гаражный пр-д : 53.1951514, 50.2609317
Координаты для  Самара Ново-Вокзальная 1: 53.2069704, 50.252648124511126
Координаты для  Сызрань Свайный п-к д1 не найдены.
Координаты для  Сызрань Смирницкой 27/29: 53.1670526, 48.4695765
Координаты для  Приволжье Волжская 4: 52.8658205, 48.602647631696314
Координаты для  Самара Солнечная 40: 53.432008100000004, 49.9157231385
Координаты для  Самара Дзержинского 48: 53.1836139, 50.1722153535074
Координаты для  Самара Гагарина 33 не найдены.
Координаты для  Самара Революционная 90: 53.4741818, 51.47935667270896
Координаты для  Самара Дыбенко 27В: 53.205265, 50.1936929
Координаты для  Самара Советской Армии 201: 53.2291933, 50.1938429
Координаты для  Самара Московское ш 106: 53.223376, 50.189667
Координаты для  Самара Ново-Садовая 44: 

In [39]:
import pandas as pd

# Прочитать обновленный файл Excel
try:
    updated_df = pd.read_excel('/content/point_updated.xlsx')
    print("Данные из обновленного файла успешно загружены.")
    print("Информация из DataFrame:")
    print(updated_df)
except Exception as e:
    print(f"Ошибка при чтении файла Excel: {e}")

Данные из обновленного файла успешно загружены.
Информация из DataFrame:
                                    Адрес     Широта    Долгота  Расстояние
0          Красный Яр сп Комсомольская 82        NaN        NaN         NaN
1                   Самара Гаражный пр-д   53.195151  50.260932    1.427070
2                Самара Ново-Вокзальная 1  53.206970  50.252648         NaN
3                  Сызрань Свайный п-к д1        NaN        NaN         NaN
4                Сызрань Смирницкой 27/29  53.167053  48.469577   34.692255
5                    Приволжье Волжская 4  52.865820  48.602648  108.110091
6                     Самара Солнечная 40  53.432008  49.915723   32.504264
7                  Самара Дзержинского 48  53.183614  50.172215         NaN
8                      Самара Гагарина 33        NaN        NaN         NaN
9                 Самара Революционная 90  53.474182  51.479357   90.714382
10                     Самара Дыбенко 27В  53.205265  50.193693    2.663008
11             